In [1]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb

import optuna
from optuna import Trial, visualization





g:\My Drive\Kaggle\tabular-playground-series-apr-2022\tab22\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
g:\My Drive\Kaggle\tabular-playground-series-apr-2022\tab22\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Tue Apr 12 11:16:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   33C    P8    20W / 250W |    936MiB /  8192MiB |     21%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
data = pd.read_csv('../Data/train_features_computed_tabular.csv')
labels = pd.read_csv('../Data/train_labels.csv')
data = data.merge(labels, on = 'sequence', how = 'left')
X, y = data.drop(['sequence', 'state'], axis = 1).values, data['state'].values

In [4]:
scaler = StandardScaler()
model=scaler.fit(X)
X=model.transform(X)

In [ ]:
def Objective(trial):

    param = {
        "tree_method": "gpu_hist",  # this parameter means using the GPU when training our model to speedup the training process
        "lambda": trial.suggest_loguniform("lambda", 1e-3, 10.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-3, 10.0),
        "colsample_bytree": trial.suggest_categorical(
            "colsample_bytree", [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        ),
        "subsample": trial.suggest_categorical(
            "subsample", [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
        ),
        "learning_rate": trial.suggest_categorical(
            "learning_rate", [0.008, 0.009, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02]
        ),
        "n_estimators": 4000,
        "max_depth": trial.suggest_categorical(
            "max_depth", [5, 7, 9, 11, 13, 15, 17, 20]
        ),
        "random_state": trial.suggest_categorical("random_state", [24, 48, 2020]),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 300),
    }

    kf = KFold(n_splits=3, random_state=42, shuffle=True)
    CV_score_array = []
    for train_index, test_index in kf.split(X):
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        clf = xgb.XGBClassifier(**param, use_label_encoder=False)
        clf.fit(
            X_train,
            y_train,
            eval_set=[(X_valid, y_valid)],
        )

        preds = clf.predict(X_valid)
        auc = roc_auc_score(y_valid, preds)

        CV_score_array.append(auc)
    avg = np.mean(CV_score_array)
    return avg


In [ ]:
study = optuna.create_study(direction="maximize", study_name='xgb optimization')
study.optimize(Objective, timeout=6*60) #5 hours

best = study.best_params
print('The best parameters are ', best)

[I 2022-04-12 11:15:32,664] A new study created in memory with name: TabNet optimization


KeyboardInterrupt: 